In [1]:
import pandas as pd
import numpy as np
HealthCondition = pd.read_csv("FinalCSVs/HealthCondition.csv")
MP = pd.read_csv("FinalCSVs/MedicalProcedure.csv")
DiseaseCase = pd.read_csv("FinalCSVs/DiseaseCase.csv")
csvs = [HealthCondition, MP]

In [2]:
for x in csvs :
    print(x.columns)

Index(['HealthCondition_ID', 'Network_ID', 'Site_ID', 'Patient_ID',
       'Encounter_ID', 'DiagnosisText_orig', 'DiagnosisText_calc',
       'DiagnosisCodeType_orig', 'DiagnosisCodeType_calc',
       'DiagnosisCode_orig', 'DiagnosisCode_calc', 'DateOfOnset',
       'SignificantNegativeFlag', 'ActiveInactiveFlag', 'DateCreated'],
      dtype='object')
Index(['MedicalProcedure_ID', 'Network_ID', 'Site_ID', 'Patient_ID',
       'Encounter_ID', 'PerformedDate', 'Name_orig', 'Name_calc',
       'DateCreated'],
      dtype='object')


In [3]:
HealthCondition2 = HealthCondition.dropna(subset=['DiagnosisCode_calc'])
HealthCondition2 = HealthCondition2[pd.to_numeric(HealthCondition2.DiagnosisCode_calc,errors='coerce').notnull()]
HealthCondition2['DiagnosisCode_calc'] = pd.to_numeric(HealthCondition2['DiagnosisCode_calc'])
 
HealthCondition2 = HealthCondition.dropna(subset=['Patient_ID'])
HealthCondition2 = HealthCondition2[pd.to_numeric(HealthCondition2.Patient_ID,errors='coerce').notnull()]
HealthCondition2['Patient_ID'] = pd.to_numeric(HealthCondition2['Patient_ID'])
print(HealthCondition2)


        HealthCondition_ID  Network_ID  Site_ID   Patient_ID  Encounter_ID  \
0                   425065           8        3  80034616133           NaN   
1                   425075           8        3  80034615671           NaN   
2                   425079           8        3  80034615616           NaN   
3                   425081           8        3  80034615586           NaN   
4                   425145           8        3  80034612993           NaN   
...                    ...         ...      ...          ...           ...   
174955              493309           8       65  80653355685           NaN   
174956              493329           8       65  80653355388           NaN   
174957              493347           8       65  80653355149           NaN   
174958              493358           8       65  80653357434           NaN   
174959              493374           8       65  80653357166           NaN   

       DiagnosisText_orig                                 Diagn

In [4]:
## all patients with diabetes in health condition
diabetesPatients = []
for index, row in HealthCondition2.iterrows() :
    patID = int(row['Patient_ID'])
    try:  
        ICD9 = float(row['DiagnosisCode_calc'])
        if (250 <= ICD9 < 251):
            diabetesPatients.append(patID)
    except:
        ICD9 = str(row['DiagnosisCode_calc'])
        pass
    
print(diabetesPatients)

[80034607647, 80034804787, 80013236545, 80013236411, 80034824307, 80034824058, 80034596264, 80034596202, 80034595298, 80034864081, 80034859733, 80034824694, 80034823984, 80613279400, 80013278531, 80034896852, 80034893850, 80034721695, 80034687314, 80034579416, 80034750952, 80013264568, 80013347968, 80034822222, 80034821550, 80613334654, 80013227536, 80034816949, 80034583577, 80013346398, 80013314227, 80613312156, 80034575240, 80034575040, 80034632660, 80034658770, 80034726407, 80034769204, 80034617152, 80613218387, 80034579490, 80034630543, 80034630088, 80034630037, 80034704094, 80034635810, 80034635736, 80034634449, 80034904578, 80034903672, 80013301321, 80013226470, 80613275862, 80034870580, 80034640348, 80034638912, 80034615616, 80034840706, 80034791445, 80034806942, 80034635217, 80034616133, 80034581611, 80034878827, 80034775020, 80034734314, 80034592154, 80034590380, 80034688654, 80034676116, 80034906897, 80034696074, 80044261852, 80034831308, 80034821765, 80034829468, 80034630724

In [5]:
## dictionary of conditions, key = ICD9 code, values = patient id 
dictOfCond = {}
for index, row in HealthCondition2.iterrows() :
    temp = str(row['Patient_ID'])
    try: 
        ICD_code = str(row['DiagnosisCode_calc'])
        if ICD_code not in dictOfCond.keys() :
            dictOfCond[ICD_code] = []
        dictOfCond[ICD_code].append(temp)
        
    except: 
        temp not in diabetesPatients
        pass   
print(dictOfCond)

{'401': ['80034616133', '80034615671', '80034828517', '80034608802', '80034607647', '80034596064', '80034583828', '80034909205', '80013262464', '80034623772', '80034804787', '80034858029', '80034825711', '80034824694', '80034596415', '80034620477', '80034620077', '80034590012', '80034892508', '80034579490', '80034795440', '80034748314', '80613256053', '80613265628', '80013264421', '80013264236', '80013248262', '80034820967', '80034664133', '80034660759', '80613221644', '80034589493', '80034817306', '80034813492', '80034670817', '80613317372', '80034830004', '80034603790', '80034646691', '80034907186', '80034907135', '80013218982', '80013218750', '80013218567', '80013217825', '80613344529', '80034631892', '80034629590', '80034588625', '80034707836', '80034808614', '80013280118', '80034906986', '80034906897', '80034906029', '80034868993', '80013218144', '80034669613', '80034697446', '80034696074', '80034615586', '80034630426', '80034611489', '80034652286', '80034836152', '80034609855', '

In [6]:
import decimal
import numpy as np

In [7]:
# AP = AnginaPectoris 413.0 - 413.9 
# AS = Atherosclerosis 440.0 - 440.9
#ICHD = Ischemic Heart Disease 410 - 414, excluding 413

AP = np.arange(413.0, 414.0, 0.1)
AS = np.arange(440.0, 441.0, 0.01)
ICHD = np.arange(410.0, 415.0, 0.1)
DD = [311]
DNP = np.arange(585.0, 586.0, 0.1)
#DNP.append(250.4)
DNU = np.arange(357.0, 358.0, 0.1)
#DNU.append(250.6)
DR = np.arange(362.0, 362.1, 0.1)
HL = [389.9]
MI = np.arange(410.0, 411.0, 0.1)
PVD = np.arange(443.0, 444.0, 0.1)
#PVD.append(250.7)

import csv
with open("APpat.csv", 'w') as a, open("ASpat.csv", 'w') as b, open("ICHDpat.csv", 'w') as c:
    
    for key in dictOfCond.keys():
        try:
            if float(key) in AP:
                #print(key)
                for i in dictOfCond[key]:
                    a.write("%s,%s\n"%(key,i))
            if float(key) in AS: 
                #print(key)
                for i in dictOfCond[key]:
                    b.write("%s,%s\n"%(key,i))
            if float(key) in ICHD: 
                print(key)
                for i in dictOfCond[key]:
                    c.write("%s,%s\n"%(key,i))
            
        except:
            pass
            #print(key)

    for i in dictOfCond["413.9"]:
        a.write("%s,%s\n"%(413.9,i))
    for i in dictOfCond["440.2"]:
        b.write("%s,%s\n"%(440.2,i))
    for i in dictOfCond["440.29"]:
        b.write("%s,%s\n"%(440.29,i))
    for i in dictOfCond["440.1"]:
        b.write("%s,%s\n"%(440.1,i))
    for i in dictOfCond["440.8"]:
        b.write("%s,%s\n"%(440.8,i))
    for i in dictOfCond["440.9"]:
        b.write("%s,%s\n"%(440.9,i))
    for i in dictOfCond["440.21"]:
        b.write("%s,%s\n"%(440.21,i))
    for i in dictOfCond["410.9"]:
        c.write("%s,%s\n"%(410.9,i))
    for i in dictOfCond["410.71"]:
        c.write("%s,%s\n"%(410.71,i))
    for i in dictOfCond["410.51"]:
        c.write("%s,%s\n"%(410.51,i))
    for i in dictOfCond["410.40"]:
        c.write("%s,%s\n"%(410.40,i))
    for i in dictOfCond["410.80"]:
        c.write("%s,%s\n"%(410.8,i))
    for i in dictOfCond["410.60"]:
        c.write("%s,%s\n"%(410.6,i))
    for i in dictOfCond["410.91"]:
        c.write("%s,%s\n"%(410.91,i))
    for i in dictOfCond["410.42"]:
        c.write("%s,%s\n"%(410.42,i))
    for i in dictOfCond["410.70"]:
        c.write("%s,%s\n"%(410.7,i))
    for i in dictOfCond["411.81"]:
        c.write("%s,%s\n"%(411.81,i))
    for i in dictOfCond["411"]:
        c.write("%s,%s\n"%(411,i))
    for i in dictOfCond["411.89"]:
        c.write("%s,%s\n"%(411.89,i))
    for i in dictOfCond["411.1"]:
        c.write("%s,%s\n"%(411.1,i))
    for i in dictOfCond["411.0"]:
        c.write("%s,%s\n"%(411.0,i))
    for i in dictOfCond["411.0"]:
        c.write("%s,%s\n"%(411.0,i))
    for i in dictOfCond["412"]:
        c.write("%s,%s\n"%(412,i))
    for i in dictOfCond["414.06"]:
        c.write("%s,%s\n"%(414.06,i))
    for i in dictOfCond["414.10"]:
        c.write("%s,%s\n"%(414.10,i))
    for i in dictOfCond["414.2"]:
        c.write("%s,%s\n"%(414.2,i))
    for i in dictOfCond["414.4"]:
        c.write("%s,%s\n"%(414.4,i))
    for i in dictOfCond["414.19"]:
        c.write("%s,%s\n"%(414.19,i))
    for i in dictOfCond["414.1"]:
        c.write("%s,%s\n"%(414.1,i))
    for i in dictOfCond["414.11"]:
        c.write("%s,%s\n"%(414.11,i))
    for i in dictOfCond["414.00"]:
        c.write("%s,%s\n"%(414.00,i))
    for i in dictOfCond["414.8"]:
        c.write("%s,%s\n"%(414.8,i))
    for i in dictOfCond["414.01"]:
        c.write("%s,%s\n"%(414.01,i))
    for i in dictOfCond["414.9"]:
        c.write("%s,%s\n"%(414.9,i))
    for i in dictOfCond["414.0"]:
        c.write("%s,%s\n"%(414.0,i))
    for i in dictOfCond["414.05"]:
        c.write("%s,%s\n"%(414.05,i))
    for i in dictOfCond["414"]:
        c.write("%s,%s\n"%(414,i))

410
410.0
410.00
410.1


In [8]:
#DD = Depressive Disorder 311
#DNP = Chronic Kidney Diseases and stuff - Nephropathy 585.0-585.9, append 250.4 
#DNU  = Neuropathy in Diabetes (Nervous System) 357.0 to 357.9, append 250.6
DD = [311]
DNP = np.arange(585.0, 586.0, 0.1)
#DNP.append(250.4)
DNU = np.arange(357.0, 358.0, 0.1)
#DNU.append(250.6)

import csv
with open("DDpat.csv", 'w') as d,open("DNPpat.csv", 'w') as e, open("DNUpat.csv", 'w') as f:
     for key in dictOfCond.keys():
        try:
            if float(key) in DD:
                #print(key)
                for i in dictOfCond[key]:
                    d.write("%s,%s\n"%(key,i))
            if float(key) in DNP:
                print(key)
                for i in dictOfCond[key]:
                    e.write("%s,%s\n"%(key,i))
            if float(key) in DNU:
                print(key)
                for i in dictOfCond[key]:
                    f.write("%s,%s\n"%(key,i))
        except:
            pass
        for i in dictOfCond["311"]:
            d.write("%s,%s\n"%(311,i))
        for i in dictOfCond["585.5"]:
            e.write("%s,%s\n"%(585.5,i))
        for i in dictOfCond["585.6"]:
            e.write("%s,%s\n"%(585.6,i)) 
        for i in dictOfCond["585.4"]:
            e.write("%s,%s\n"%(585.4,i)) 
        for i in dictOfCond["585.9"]:
            e.write("%s,%s\n"%(585.9,i)) 
        for i in dictOfCond["585.3"]:
            e.write("%s,%s\n"%(585.3,i)) 
        for i in dictOfCond["250.42"]:
            e.write("%s,%s\n"%(250.42,i))
        for i in dictOfCond["250.40"]:
            e.write("%s,%s\n"%(250.40,i))
        for i in dictOfCond["250.4"]:
            e.write("%s,%s\n"%(250.4,i)) 
        for i in dictOfCond["250.43"]:
            e.write("%s,%s\n"%(250.43,i)) 
        for i in dictOfCond["357.89"]:
            f.write("%s,%s\n"%(357.89,i))
        for i in dictOfCond["357.3"]:
            f.write("%s,%s\n"%(357.3,i)) 
        for i in dictOfCond["357.8"]:
            f.write("%s,%s\n"%(357.8,i)) 
        for i in dictOfCond["357.82"]:
            f.write("%s,%s\n"%(357.82,i)) 
        for i in dictOfCond["357.5"]:
            f.write("%s,%s\n"%(357.5,i)) 
        for i in dictOfCond["357.9"]:
            f.write("%s,%s\n"%(357.9,i))
        for i in dictOfCond["357.6"]:
            f.write("%s,%s\n"%(357.6,i)) 
        for i in dictOfCond["357.81"]:
            f.write("%s,%s\n"%(357.81,i))
        for i in dictOfCond["357.4"]:
            f.write("%s,%s\n"%(357.4,i)) 
        for i in dictOfCond["357.2"]:
            f.write("%s,%s\n"%(357.2,i)) 
        for i in dictOfCond["250.62"]:
            f.write("%s,%s\n"%(250.62,i)) 
        for i in dictOfCond["250.61"]:
            f.write("%s,%s\n"%(250.61,i))
        for i in dictOfCond["250.60"]:
            f.write("%s,%s\n"%(250.60,i))
        for i in dictOfCond["250.6"]:
            f.write("%s,%s\n"%(250.6,i))


585
585.1
585.2
357
357.0


In [9]:
#DR = Diabetic Retinopathy, 362.0 to 362.10
#HL = Hearing Loss
#MI = Acute myocardial infarction, 410 - 410.9
#PVD = Peripheral vascular disease, 443.0 to 443.9, append 250.7

DR = np.arange(362.0, 362.1, 0.1)
HL = [389.9]
MI = np.arange(410.0, 411.0, 0.1)
PVD = np.arange(443.0, 444.0, 0.1)
#PVD.append(250.7)
import csv 
with open("DRpat.csv", 'w') as g, open("HLpat.csv", 'w') as h, open("MIpat.csv", 'w') as m, open("PVDpat.csv", 'w') as j:
    for key in dictOfCond.keys():
        try:
            if float(key) in DR:
                #print(key)
                for i in dictOfCond[key]:
                    g.write("%s,%s\n"%(key,i))
            if float(key) in HL:
                #print(key)
                for i in dictOfCond[key]:
                    h.write("%s,%s\n"%(key,i))
            if float(key) in MI:
                #print(key)
                for i in dictOfCond[key]:
                    m.write("%s,%s\n"%(key,i))
            if float(key) in PVD:
                print(key)
                for i in dictOfCond[key]:
                    j.write("%s,%s\n"%(key,i))
        except:
            pass
    for i in dictOfCond["362.03"]:
            g.write("%s,%s\n"%(362.03,i))
    for i in dictOfCond["362.04"]:
            g.write("%s,%s\n"%(362.04,i))
    for i in dictOfCond["362.06"]:
            g.write("%s,%s\n"%(362.06,i))
    for i in dictOfCond["362.07"]:
            g.write("%s,%s\n"%(362.07,i))
    for i in dictOfCond["362.02"]:
            g.write("%s,%s\n"%(362.02,i))
    for i in dictOfCond["362.01"]:
            g.write("%s,%s\n"%(362.01,i))
    for i in dictOfCond["410.9"]:
            m.write("%s,%s\n"%(410.9,i))
    for i in dictOfCond["410.71"]:
            m.write("%s,%s\n"%(410.71,i))
    for i in dictOfCond["410.51"]:
            m.write("%s,%s\n"%(410.51,i))
    for i in dictOfCond["410.40"]:
            m.write("%s,%s\n"%(410.40,i))
    for i in dictOfCond["410.80"]:
            m.write("%s,%s\n"%(410.8,i))
    for i in dictOfCond["410.60"]:
            m.write("%s,%s\n"%(410.6,i))
    for i in dictOfCond["410.91"]:
            m.write("%s,%s\n"%(410.91,i))
    for i in dictOfCond["410.42"]:
            m.write("%s,%s\n"%(410.42,i))
    for i in dictOfCond["410.70"]:
            m.write("%s,%s\n"%(410.7,i))
    for i in dictOfCond["443.23"]:
            j.write("%s,%s\n"%(443.23,i))
    for i in dictOfCond["443.22"]:
            j.write("%s,%s\n"%(443.22,i))
    for i in dictOfCond["443.82"]:
            j.write("%s,%s\n"%(443.82,i))
    for i in dictOfCond["443.24"]:
            j.write("%s,%s\n"%(443.24,i))
    for i in dictOfCond["443.21"]:
            j.write("%s,%s\n"%(443.21,i))
    for i in dictOfCond["443.29"]:
            j.write("%s,%s\n"%(443.29,i))
    for i in dictOfCond["443.8"]:
            j.write("%s,%s\n"%(443.8,i))
    for i in dictOfCond["443.89"]:
            j.write("%s,%s\n"%(443.89,i))
    for i in dictOfCond["443.9"]:
            j.write("%s,%s\n"%(443.9,i))
    for i in dictOfCond["250.7"]:
            j.write("%s,%s\n"%(250.7,i))
    for i in dictOfCond["250.70"]:
            j.write("%s,%s\n"%(250.70,i))

443
443.0


In [38]:
#run for individual csvs for every single ICD9 code (5180 files)
import csv
for key in dictOfCond.keys():
    with open(key+'test.csv', 'w') as f:
        for i in dictOfCond[key]:
            f.write("%s,%s\n"%(key,i))